In [1]:
import glob
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date
from datetime import datetime

D:\programming\Anaconda1\envs\NN\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt
import cv2

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 8, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((8, 8, 256)))
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(512, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(256, filter_size_g, strides=(1,1), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(128, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(16, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(8, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(1,1), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [ ]:
def build_discriminator():

        model = Sequential()
        
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.summary()

        return model

In [ ]:
def generator_containing_discriminator(g, d):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    #d_label.trainable = False
    valid = d(img)
    
    
    return Model(noise, valid)

In [ ]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [ ]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [ ]:
def get_images_classes(batch_size, data):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        rand_number = np.random.randint(0, len(data[choice_arr[i]]))
        temp_img = cv2.imread(data[choice_arr[i]][rand_number])
        
        
        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size) + class_target
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(batch_size):
        temp_img = cv2.imread(os.getcwd() + '/new256_images/abstraktnyy-ekspressionizm/303.png')
        
        X_train[i] = temp_img
        y_labels[i] = 0
    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [ ]:
def get_data():
    styles_folder = os.listdir(path=os.getcwd() + "\\new256_images\\")
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data.append(glob.glob(os.getcwd() + '\\new256_images\\' + styles_folder[i] + '\\*'))
    return data, num_styles

In [ ]:
def train_another(epochs = 100, BATCH_SIZE = 4):

    data, num_styles = get_data()
    
    generator = build_generator()
    discriminator = build_discriminator()
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    #d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    dcgan = generator_containing_discriminator(generator, discriminator)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    #d_label.trainable = True
    
    
    '''(X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))'''
    
    
    for epoch in range(epochs):
        for index in range(int(15000/BATCH_SIZE)):
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            label = index % num_styles
            
            real_images, real_labels = get_images_one_class(BATCH_SIZE, data, label)
            
            real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + label, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                #discriminator.trainable = False
                #d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss[0]))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            
            
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            #d_label.trainable = False
            
            y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 0.5, num_styles)
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, y)
            
            #d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss : %f" % (epoch, index, g_loss))
            
            if index % 50 == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, index), image)
                        image = combine_images(real_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            '''# Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()'''
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '%d.%d dis_weights.h5' % (day, month))
            #d_label.save_weights(os.getcwd() + '%d.%d dis_label_weights.h5' % (day, month))
            generator.save_weights(os.getcwd() + '%d.%d gen_weights.h5' % (day, month))

In [ ]:
img_rows = 256
img_cols = 256
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.001, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.00025, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.5)
d_optim = Adam(0.0002, beta_2 = 0.5)

In [ ]:
train_another(1000, 16)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16384)             1654784   
_________________________________________________________________
batch_normalization_1 (Batch (None, 16384)             65536     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16384)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 8, 8, 512)         3277312   
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 512)         2048      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 512)         0         
__________

epoch 0 batch 2 d_loss : 1.581627
epoch 0 batch 2 g_loss : 0.421538
epoch 0 batch 3 d_loss : 0.412669
epoch 0 batch 3 g_loss : 0.918834
epoch 0 batch 4 d_loss : 0.931082
epoch 0 batch 4 g_loss : 1.284591
epoch 0 batch 5 d_loss : 0.875572
epoch 0 batch 5 g_loss : 1.106225
epoch 0 batch 6 d_loss : 2.418252
epoch 0 batch 6 g_loss : 1.749577
epoch 0 batch 7 d_loss : 0.935993
epoch 0 batch 7 g_loss : 1.521374
epoch 0 batch 8 d_loss : 0.632271
epoch 0 batch 8 g_loss : 0.788820
epoch 0 batch 9 d_loss : 0.491100
epoch 0 batch 9 g_loss : 0.674360
epoch 0 batch 10 d_loss : 1.509620
epoch 0 batch 10 g_loss : 1.838554
epoch 0 batch 11 d_loss : 2.367131
epoch 0 batch 11 g_loss : 1.214953
epoch 0 batch 12 d_loss : 1.590645
epoch 0 batch 12 g_loss : 0.568986
epoch 0 batch 13 d_loss : 0.469259
epoch 0 batch 13 g_loss : 0.861941
epoch 0 batch 14 d_loss : 1.282153
epoch 0 batch 14 g_loss : 1.654452
epoch 0 batch 15 d_loss : 1.000771
epoch 0 batch 15 g_loss : 1.873090
epoch 0 batch 16 d_loss : 1.248353
e

epoch 0 batch 119 d_loss : 0.360028
epoch 0 batch 119 g_loss : 0.475239
epoch 0 batch 120 d_loss : 0.331618
epoch 0 batch 120 g_loss : 0.597942
epoch 0 batch 121 d_loss : 0.448542
epoch 0 batch 121 g_loss : 0.632733
epoch 0 batch 122 d_loss : 0.334933
epoch 0 batch 122 g_loss : 0.559499
epoch 0 batch 123 d_loss : 0.432462
epoch 0 batch 123 g_loss : 0.423515
epoch 0 batch 124 d_loss : 0.312176
epoch 0 batch 124 g_loss : 0.510768
epoch 0 batch 125 d_loss : 0.290187
epoch 0 batch 125 g_loss : 0.416084
epoch 0 batch 126 d_loss : 0.380018
epoch 0 batch 126 g_loss : 0.545880
epoch 0 batch 127 d_loss : 0.362629
epoch 0 batch 127 g_loss : 0.610396
epoch 0 batch 128 d_loss : 0.374493
epoch 0 batch 128 g_loss : 0.466003
epoch 0 batch 129 d_loss : 0.496681
epoch 0 batch 129 g_loss : 0.667060
epoch 0 batch 130 d_loss : 0.366042
epoch 0 batch 130 g_loss : 0.544723
epoch 0 batch 131 d_loss : 0.436250
epoch 0 batch 131 g_loss : 0.531642
epoch 0 batch 132 d_loss : 0.406847
epoch 0 batch 132 g_loss : 0

epoch 0 batch 233 d_loss : 0.384767
epoch 0 batch 233 g_loss : 0.511478
epoch 0 batch 234 d_loss : 0.318527
epoch 0 batch 234 g_loss : 0.455358
epoch 0 batch 235 d_loss : 0.361590
epoch 0 batch 235 g_loss : 0.440838
epoch 0 batch 236 d_loss : 0.332525
epoch 0 batch 236 g_loss : 0.455800
epoch 0 batch 237 d_loss : 0.349816
epoch 0 batch 237 g_loss : 0.405976
epoch 0 batch 238 d_loss : 0.355179
epoch 0 batch 238 g_loss : 0.437939
epoch 0 batch 239 d_loss : 0.341172
epoch 0 batch 239 g_loss : 0.425106
epoch 0 batch 240 d_loss : 0.259941
epoch 0 batch 240 g_loss : 0.467467
epoch 0 batch 241 d_loss : 0.346637
epoch 0 batch 241 g_loss : 0.406927
epoch 0 batch 242 d_loss : 0.348409
epoch 0 batch 242 g_loss : 0.405305
epoch 0 batch 243 d_loss : 0.372341
epoch 0 batch 243 g_loss : 0.446771
epoch 0 batch 244 d_loss : 0.292736
epoch 0 batch 244 g_loss : 0.416150
epoch 0 batch 245 d_loss : 0.295105
epoch 0 batch 245 g_loss : 0.505306
epoch 0 batch 246 d_loss : 0.308373
epoch 0 batch 246 g_loss : 0

In [ ]:
current_moment = datetime.now()

day, month = str(current_moment.day), str(current_moment.month)
hour, minute = str(current_moment.hour), str(current_moment.minute)

with open(os.getcwd() + '/Changes.txt', 'a') as f:
    clock =  (day + '.' + month + '    ' + hour + ':' + minute)
    f.write(clock + '...........g_optim -> 0.0003, d_optim->0.0004' + '\n')